In [1]:
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D
import os 
import pandas as pd
import numpy as np

In [2]:
TRAIN_DIR='images/train'
TEST_DIR='images/test'


In [3]:
 def createdataframe(dir):
     image_paths = []
     labels =[]
     for label in os.listdir(dir):
         for imagename in os.listdir(os.path.join(dir,label)):
             image_paths.append(os.path.join(dir,label,imagename))
             labels.append(label)
         print (label,"completed")
     return image_paths,labels

In [4]:
train=pd.DataFrame()
train['image'],train['label']=createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)

                                image     label
0         images/train\angry\3386.jpg     angry
1         images/train\angry\3387.jpg     angry
2        images/train\angry\33889.jpg     angry
3        images/train\angry\33892.jpg     angry
4        images/train\angry\33897.jpg     angry
...                               ...       ...
25889  images/train\surprise\9969.jpg  surprise
25890  images/train\surprise\9985.jpg  surprise
25891  images/train\surprise\9990.jpg  surprise
25892  images/train\surprise\9992.jpg  surprise
25893  images/train\surprise\9996.jpg  surprise

[25894 rows x 2 columns]


In [6]:
test=pd.DataFrame()
test['image'],test['label']=createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)

Empty DataFrame
Columns: [image, label]
Index: []


In [8]:
from tqdm.notebook import tqdm

In [9]:
def extract_features(images):
    features =[]
    for image in tqdm(images):
        img = load_img(image, color_mode="grayscale", target_size=(48,48))
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features

In [10]:
train_features=extract_features(train['image'])

  0%|          | 0/25894 [00:00<?, ?it/s]

In [14]:
test_features=extract_features(test['image'])

0it [00:00, ?it/s]

In [15]:
x_train=train_features/255.0
x_test=test_features/255.0

In [16]:
from sklearn.preprocessing import LabelEncoder 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout


In [17]:
le=LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train= le.transform(train['label'])
y_test=le.transform(test['label'])

In [19]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [20]:
model=Sequential()
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))


model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))


model.add(Dense(7,activation='softmax'))


C:\Users\khoul\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(x=x_train,y=y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
 17/203 ━━━━━━━━━━━━━━━━━━━━ 5:10 2s/step - accuracy: 0.2369 - loss: 1.8552

KeyboardInterrupt: 

In [ ]:
model_json=model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file=open("facialemotionmodel.json","r")
model_json=json_file.read()
json_file.close()
model=model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")


In [ ]:
label=['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
      img = load_img(image, color_mode="grayscale", target_size=(48,48))
      img=np.array(img)
      img=img.reshape(1,48,48,1)
      return img/255.0

In [ ]:
image='images/train/sad/42.jpg'
print("original image is of sad")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("model prediction is ",pred_label)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
image='images/train/sad/42.jpg'
print("original image is of sad")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='grey')